In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime 
tf.set_random_seed(77)



In [2]:
forecastDay=7
rawArrayDatas=[["2017-08-02",
"2017-08-03",
"2017-08-04",
"2017-08-05",
"2017-08-06",
"2017-08-07",
"2017-08-08",
"2017-08-09",
"2017-08-10",
"2017-08-11",
"2017-08-12",
"2017-08-13",
"2017-08-14",
"2017-08-15",
"2017-08-16",
"2017-08-17",
"2017-08-18",
"2017-08-19",
"2017-08-20",
"2017-08-21",
"2017-08-22",
"2017-08-23",
"2017-08-24",
"2017-08-25",
"2017-08-26",
"2017-08-27",
"2017-08-28",
"2017-08-29",
"2017-08-30",
"2017-08-31",
"2017-09-01",
"2017-09-02",
"2017-09-03",
"2017-09-04",
"2017-09-05",
"2017-09-06",
"2017-09-07",
"2017-09-08",
"2017-09-09",
"2017-09-10",
"2017-09-11",
"2017-09-12",
"2017-09-13",
"2017-09-14",
"2017-09-15",
"2017-09-16",
"2017-09-17",
"2017-09-18",
"2017-09-19",
"2017-09-20",
"2017-09-21",
"2017-09-22",
"2017-09-23",
"2017-09-24",
"2017-09-25",
"2017-09-26",
"2017-09-27",
"2017-09-28",
"2017-09-29",
"2017-09-30",
"2017-10-01",
"2017-10-02",
"2017-10-03",
"2017-10-04",
"2017-10-05",
"2017-10-06",
"2017-10-07",
"2017-10-08",
"2017-10-09",
"2017-10-10",
"2017-10-11",
"2017-10-12",
"2017-10-13",
"2017-10-14",
"2017-10-15",
"2017-10-16",
"2017-10-17",
"2017-10-18",
"2017-10-19",
"2017-10-20",
"2017-10-21",
"2017-10-22",
"2017-10-23"],[34,
41,
54,
41,
35,
44,
50,
42,
42,
66,
50,
55,
56,
53,
44,
54,
54,
50,
40,
49,
28,
72,
71,
53,
43,
38,
55,
49,
43,
49,
49,
44,
39,
52,
45,
33,
43,
40,
46,
49,
50,
37,
37,
45,
48,
48,
38,
60,
31,
35,
53,
70,
62,
48,
51,
49,
38,
32,
39,
35,
30,
36,
31,
31,
44,
41,
41,
45,
46,
45,
41,
47,
48,
40,
42,
38,
38,
45,
48,
62,
46,
38,
62,
81]]

In [3]:
def LSTM(ds, y, forecastDay):
    sales = list(zip(ds, y))
    txs = pd.DataFrame(data=sales, columns=['date', 'sales'])
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    # please read docs on how week numbers are calculate
    week_number = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')

    txs['year'] = txs['date'].map(year)
    txs['month'] = txs['date'].map(month)
    txs['week_number'] = txs['date'].map(week_number)
    txs['day_of_week'] = txs['date'].map(day_of_week)

    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['date'].map(season)
    txs['day_of_week01'] = txs['date'].map(day_of_week01)
    originalSales = list(txs['sales'])
    sales = list(txs['sales'])

    tempxy = [list(txs['season']), list(txs['day_of_week']), list(txs['week_number']), sales]

    xy = np.array(tempxy).transpose().astype(np.float)
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 4

    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 5

    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay

    # hidden_dim은 정말 임의로 설정
    hidden_dim = 10

    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.01

    # iterations는 반복 횟수
    iterations = 1000

    x = xy
    y = xy[:, [-1]]

    # build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        #     _y=Y[i+seq_length:i+seq_length+forecastDays]
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size
    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])
    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.nn.relu)
#     cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_units=hidden_dim, activation=tf.nn.relu)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)

    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares

    optimizer = tf.train.AdamOptimizer(learning_rate)

    train = optimizer.minimize(loss)

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
#     rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
    with tf.Session() as sess:
        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):
            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            # print("[step: {}] loss: {}".format(i, step_loss))

        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)

    return list(test_predict[-1])



In [4]:
mockForecastDictionary={}
realForecastDictionary={}
def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier


In [5]:

global mockForecastDictionary
global realForecastDictionary
trainSize=int(len(rawArrayDatas[0])*0.7)
testSize=len(rawArrayDatas[0])-trainSize

testY= rawArrayDatas[1][trainSize:]

####################################################################################LSTM

testY= rawArrayDatas[1][trainSize:]
mockDs = rawArrayDatas[0][:trainSize]
mockY = list((rawArrayDatas[1][:trainSize]))
mockSales = list(zip(mockDs, mockY))

ds = rawArrayDatas[0]
y = list(rawArrayDatas[1])
sales = list(zip(ds, y))

mockForecastDictionary['LSTM']= LSTM(mockDs, mockY, testSize)
tf.reset_default_graph()
realForecastDictionary['LSTM'] = LSTM(ds,y, forecastDay)


[[ 0.02777778  0.02777778  0.43055555  0.47222222]
 [ 0.02777778  0.04166667  0.43055555  0.56944444]
 [ 0.02777778  0.05555556  0.43055555  0.75      ]
 [ 0.02777778  0.06944444  0.43055555  0.56944444]
 [ 0.02777778  0.08333333  0.43055555  0.48611111]] -> [ 0.61111111  0.69444444  0.58333333  0.58333333  0.91666667  0.69444444
  0.76388889  0.77777778  0.73611111  0.61111111  0.75        0.75
  0.69444444  0.55555555  0.68055555  0.38888889  1.          0.98611111
  0.73611111  0.59722222  0.52777778  0.76388889  0.68055555  0.59722222
  0.68055555]
[[ 0.02777778  0.04166667  0.43055555  0.56944444]
 [ 0.02777778  0.05555556  0.43055555  0.75      ]
 [ 0.02777778  0.06944444  0.43055555  0.56944444]
 [ 0.02777778  0.08333333  0.43055555  0.48611111]
 [ 0.02777778  0.          0.44444444  0.61111111]] -> [ 0.69444444  0.58333333  0.58333333  0.91666667  0.69444444  0.76388889
  0.77777778  0.73611111  0.61111111  0.75        0.75        0.69444444
  0.55555555  0.68055555  0.38888889

[[ 0.02777778  0.02777778  0.43055555  0.47222222]
 [ 0.02777778  0.04166667  0.43055555  0.56944444]
 [ 0.02777778  0.05555556  0.43055555  0.75      ]
 [ 0.02777778  0.06944444  0.43055555  0.56944444]
 [ 0.02777778  0.08333333  0.43055555  0.48611111]] -> [ 0.61111111  0.69444444  0.58333333  0.58333333  0.91666667  0.69444444
  0.76388889]
[[ 0.02777778  0.04166667  0.43055555  0.56944444]
 [ 0.02777778  0.05555556  0.43055555  0.75      ]
 [ 0.02777778  0.06944444  0.43055555  0.56944444]
 [ 0.02777778  0.08333333  0.43055555  0.48611111]
 [ 0.02777778  0.          0.44444444  0.61111111]] -> [ 0.69444444  0.58333333  0.58333333  0.91666667  0.69444444  0.76388889
  0.77777778]
[[ 0.02777778  0.05555556  0.43055555  0.75      ]
 [ 0.02777778  0.06944444  0.43055555  0.56944444]
 [ 0.02777778  0.08333333  0.43055555  0.48611111]
 [ 0.02777778  0.          0.44444444  0.61111111]
 [ 0.02777778  0.01388889  0.44444444  0.69444444]] -> [ 0.58333333  0.58333333  0.91666667  0.69444444 

 [ 0.04166667  0.06944444  0.54166667  0.48611111]] -> [ 0.41666667  0.5         0.43055555  0.43055555  0.61111111  0.56944444
  0.56944444]
[[ 0.04166667  0.02777778  0.54166667  0.52777778]
 [ 0.04166667  0.04166667  0.54166667  0.44444444]
 [ 0.04166667  0.05555556  0.54166667  0.54166667]
 [ 0.04166667  0.06944444  0.54166667  0.48611111]
 [ 0.04166667  0.08333333  0.54166667  0.41666667]] -> [ 0.5         0.43055555  0.43055555  0.61111111  0.56944444  0.56944444
  0.625     ]
[[ 0.04166667  0.04166667  0.54166667  0.44444444]
 [ 0.04166667  0.05555556  0.54166667  0.54166667]
 [ 0.04166667  0.06944444  0.54166667  0.48611111]
 [ 0.04166667  0.08333333  0.54166667  0.41666667]
 [ 0.04166667  0.          0.55555555  0.5       ]] -> [ 0.43055555  0.43055555  0.61111111  0.56944444  0.56944444  0.625
  0.63888889]
[[ 0.04166667  0.05555556  0.54166667  0.54166667]
 [ 0.04166667  0.06944444  0.54166667  0.48611111]
 [ 0.04166667  0.08333333  0.54166667  0.41666667]
 [ 0.04166667  0. 

In [6]:
mockForecastDictionary

{'LSTM': [51.868969,
  52.786411,
  50.909863,
  48.595196,
  46.352955,
  48.01627,
  49.648212,
  49.896049,
  50.786079,
  47.176044,
  45.526066,
  43.52877,
  44.494629,
  45.985256,
  43.960884,
  45.014832,
  41.562935,
  42.065407,
  42.854336,
  42.89249,
  42.53907,
  40.504341,
  42.383091,
  43.890247,
  45.01453]}

In [7]:
realForecastDictionary

{'LSTM': [45.141876,
  46.630505,
  43.791889,
  46.5961,
  47.455799,
  46.812469,
  44.036293]}